In [470]:
import requests
import numpy as np 
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv

In [471]:
# Lecture du code barre
#
#

bar_code = 3608580823452 # 3116740033875 5449000214799 0037600106009 8000500310427 8076809529419 3608580823452 737628064502 3228021170060 8000430138719 3523230029681

In [472]:
# Intérrogation de l'API 'OpenfoodFacts'
url = f"https://world.openfoodfacts.org/api/v2/product/{bar_code}.json" 

answer = requests.get(url)
print(answer)                # si <Response [200]> = transaction ok

product = answer.json()
product_info = product['product']

<Response [200]>


<div class="alert alert-success" align='center'>
-> Si la réponse est 200, le lien avec l'API est OK.
</div>

In [473]:
try:
    print(f"Produit: {product_info['product_name']}")
except:
    None

Produit: Confiture Abricot INTENSE


In [474]:
print(f"Origine: {product_info['origins']}")

Origine: 


<div class="alert alert-danger" align='center'>
-> Parfois il n'y a pas d'origine car il y a plusieurs sites de fabrication pour un même produit. <br> Ex: Il y a aujourd'hui 868 usines de Coca-Cola en dehors des Etats-Unis.
</div>

In [475]:
score_transport_fr = product_info['ecoscore_data']['adjustments']['origins_of_ingredients']['transportation_scores']['fr']
print(f"score transport: {score_transport_fr}")


score transport: 0


<div class="alert alert-warning" align='center'>
-> A nuancer. <br> Comment est-ce calculé? <br> Comment est-ce calculé quand l'origine est inconnue?
</div>

<div class="alert alert-success" align='center'>
Quand elle existe, l'origine permet de faire le lien avec les autres bases de données pour le calcul de l'impact environnemental.
</div>

In [476]:
product_compounds = [compound['id'][3:] for compound in product_info['ingredients']]
print(f"composition: {product_compounds}")


composition: ['apricot', 'sugar', 'concentrated-lemon-juice', 'fruit-pectin']


In [477]:
product_metric = int(product_info['product_quantity'])
product_quantity_unit = product_info['product_quantity_unit']

print(f"Le produit fait {product_metric}{product_quantity_unit}.")

Le produit fait 335g.


In [478]:
product_compounds_info = []
total_percent = 0
ciqual_percent = 0
ciqual_food_code_list = []
ingredient_list = []

for k in range(len(product_compounds)):
  if 'ingredients' in product_info['ingredients'][k]:

    composant = product_info['ingredients'][k]['id'][3:]
    percent = product_info['ingredients'][k]['percent_estimate']

    print(f"Composant {k+1}: {composant}")
    print(f"Part dans le produit: {percent}%")
    print("\n")


    for j in range(len(product_info['ingredients'][k]['ingredients'])):
      ingredient = product_info['ingredients'][k]['ingredients'][j]['id'][3:]
      percent = product_info['ingredients'][k]['ingredients'][j]['percent_estimate']
      total_percent += percent

      if 'ciqual_food_code' in product_info['ingredients'][k]['ingredients'][j]:
        ciqual_food_code = product_info['ingredients'][k]['ingredients'][j]['ciqual_food_code']
        ciqual_food_code_list.append(ciqual_food_code)
        ciqual_percent += percent
      elif 'ciqual_proxy_food_code' in product_info['ingredients'][k]['ingredients'][j]:
        ciqual_food_code = product_info['ingredients'][k]['ingredients'][j]['ciqual_proxy_food_code']
        ciqual_food_code_list.append(ciqual_food_code)
        ciqual_percent += percent
      else:
        ciqual_food_code = '-'

      ingredient_list.append([ingredient, ciqual_food_code, percent])


      if 'origins' in product_info['ingredients'][k]['ingredients'][j]:
        origin = product_info['ingredients'][k]['ingredients'][j]['origins'][3:]
      else: 
        origin = 'Non spécifiée'

      
      print(f"\tIngrédient {j+1}: {ingredient}")
      print(f"\tCode CIQUAL: {ciqual_food_code}")
      print(f"\tOrigine: {origin}")
      print(f"\tPart dans le produit: {percent}%")
      print("\t***************\n")

  else:
    ingredient = product_info['ingredients'][k]['id'][3:]
    percent = product_info['ingredients'][k]['percent_estimate']
    total_percent += percent

    if 'ciqual_food_code' in product_info['ingredients'][k]:
      ciqual_food_code = product_info['ingredients'][k]['ciqual_food_code']
      ciqual_food_code_list.append(ciqual_food_code)
      ciqual_percent += percent
    elif 'ciqual_proxy_food_code' in product_info['ingredients'][k]:
      ciqual_food_code = product_info['ingredients'][k]['ciqual_proxy_food_code']
      ciqual_food_code_list.append(ciqual_food_code)
      ciqual_percent += percent
    else: 
      ciqual_food_code = '-'

    ingredient_list.append([ingredient, ciqual_food_code, percent])


    if 'origins' in product_info['ingredients'][k]:
      origin = product_info['ingredients'][k]['origins'][3:]
    else: 
      origin = 'Non spécifiée'

    
    print(f"Composant {k+1}: {ingredient}")
    print(f"Code CIQUAL: {ciqual_food_code}")
    print(f"Origine: {origin}")
    print(f"Part dans le produit: {percent}%")
    print("*******************************\n")

ciqual_food_code_list = np.unique(ciqual_food_code_list).tolist()
ciqual_food_code_list_int = list(map(int, ciqual_food_code_list))

Composant 1: apricot
Code CIQUAL: 13000
Origine: Non spécifiée
Part dans le produit: 62.5%
*******************************

Composant 2: sugar
Code CIQUAL: 31016
Origine: Non spécifiée
Part dans le produit: 18.75%
*******************************

Composant 3: concentrated-lemon-juice
Code CIQUAL: 2028
Origine: Non spécifiée
Part dans le produit: 9.375%
*******************************

Composant 4: fruit-pectin
Code CIQUAL: -
Origine: Non spécifiée
Part dans le produit: 9.375%
*******************************



In [479]:
print("Vérif:")
print(f"La somme des parts des ingrédients fait bien 100%: {99 <= int(total_percent) & int(total_percent) <= 101}")
print(f"La somme des parts des ingrédients dont on connait le code CIQUAL est: {ciqual_percent}%")

Vérif:
La somme des parts des ingrédients fait bien 100%: True
La somme des parts des ingrédients dont on connait le code CIQUAL est: 90.625%


<div class="alert alert-danger" align='center'>
Si la part des composants dont le code CIQUAL n'est pas supérieure à un certain seuil (85%), il est difficile de pouvoir noter le produit avec une grande fiabilité.
</div>

In [480]:
print("Liste des codes CIQUAL renseignés présents dans le produit:")
print(ciqual_food_code_list_int)

Liste des codes CIQUAL renseignés présents dans le produit:
[13000, 2028, 31016]


<div class="alert alert-success" align='center'>
Cette liste permet de faire le lien avec les autres bases de données.
</div>

<div class="alert alert-success" align='center'>

I. Base de données Agribalyse
</div>

<div class="alert alert-danger" align='center'>


/!\ CREER UN FICHIER `.env` QUI CONTIENT LES ELEMENTS POUR SE CONNECTER A LA BASE DE DONNEES /!\
</div>

In [481]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST") #localhost
DB_NAME = os.getenv("DB_NAME") #db-agribalyse
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

conn = psycopg2.connect(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD
)

In [482]:
cur = conn.cursor()

placeholders = ','.join(['%s'] * len(ciqual_food_code_list_int))

cur.execute(f"SELECT code_agb, code_ciqual, carbon_impact FROM carbon_footprint WHERE code_ciqual IN ({placeholders})", tuple(ciqual_food_code_list_int))

results = cur.fetchall()

col_names = [desc[0] for desc in cur.description]

filtered_agribalyse = pd.DataFrame(results, columns=col_names)

cur.close()
conn.close()

filtered_agribalyse

,code_agb,code_ciqual,carbon_impact
0,2028,2028,0.786944
1,13000,13000,0.880638
2,31016,31016,0.612366


In [483]:
total_co2 = 0
detail = []

for code_agb in filtered_agribalyse['code_agb'].values:
    ciqual_code = filtered_agribalyse.loc[filtered_agribalyse['code_agb'] == code_agb, 'code_ciqual'].values[0]
    carbon_footprint = filtered_agribalyse.loc[filtered_agribalyse['code_agb'] == code_agb, 'carbon_impact'].values[0]
    mass = 0

    for ingredient, code, percent in ingredient_list:
        if code != '-':
            if int(code) == ciqual_code:
                mass = (percent/100)*product_metric

                if product_quantity_unit == 'g':
                    carbon_footprint_ingredient = carbon_footprint*mass/1000
                    total_co2 += carbon_footprint_ingredient

                elif product_quantity_unit == 'kg':
                    carbon_footprint_ingredient = carbon_footprint*mass
                    total_co2 += carbon_footprint_ingredient

                if percent != 0:
                    detail.append([ingredient, carbon_footprint_ingredient])

                
    
print(f"\t\t\t***IMPACT CARBONNE***\n\t'{product_info['product_name']}'\n\t\t\t***IMPACT CARBONNE***")
print(f"/!\ NE PREND NI EN COMPTE LE TRANSPORT NI L'EMBALLAGE /!\\")
print(f"\nOrigine: {product_info['origins']}")
print(f"TOTAL: {round(total_co2*100, 2)} gCo2\n")

print("****** DETAILS ******")
for ingredient, carbon_footprint in detail:
    print(f"\t{ingredient}: {round(carbon_footprint*100, 5)} gCo2")

print("\n\n****** INGREDIENTS EN PROPORTIONS NEGLIGEABLES ******")
for ingredient, code, percent in ingredient_list:
    if percent == 0:
        print(f"\t{ingredient}")

			***IMPACT CARBONNE***
	'Confiture Abricot INTENSE'
			***IMPACT CARBONNE***
/!\ NE PREND NI EN COMPTE LE TRANSPORT NI L'EMBALLAGE /!\

Origine: 
TOTAL: 24.76 gCo2

****** DETAILS ******
	concentrated-lemon-juice: 2.47149 gCo2
	apricot: 18.43836 gCo2
	sugar: 3.84642 gCo2


****** INGREDIENTS EN PROPORTIONS NEGLIGEABLES ******


<div class="alert alert-success" align='center'>

L'utilisateur reçoit ainsi les données sur **l'origine** du produit (quand elle est fournie) et l'impact environnemental de la **production** des ingrédients (en kg de Co2 équivalent).

</div>

<div class="alert alert-warning" align='center'>

II. Base de données transporteur   


Il faudrait maintenant refaire le travail fait avec la base de données Agribalyse sur une base de données fournie par un transpoteur pour calculer l'impact carbonne du transport.
</div>